In [1]:
import copy
import numpy as np
from collections import deque
import os.path
import shutil
import pickle
import dill

import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import random
import numpy.random
import torch.random

from datetime import datetime

from copy import deepcopy

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
# Modified from J.Y. Kim. (2020)
args.master_directory = os.getcwd()
args.basicfilepath = 'data'
args.simulation_directory = 'simulation'
args.variable_save_directory = 'variables'
args.deeplearningmodel_save_directory = 'model'
args.ecl_filename = '2D_ECL'
args.perm_filename = 'PERMX'
args.well_filename = 'WELL'

# args.total_episode = 100
args.learning_rate = 0.1  # Learning rate Alpha
# args.boltzmann_tau_start = 5.0  # Start value of Temperature parameter at Boltzmann policy, Tau
args.boltzmann_tau_start = 8.0  # Start value of Temperature parameter at Boltzmann policy, Tau
args.boltzmann_tau_end = 0.1  # End value of Temperature parameter at Boltzmann policy, Tau
args.boltzmann_tau_tracker = [args.boltzmann_tau_start]
# args.total_reward = 0
args.epsilon = 0.1

# For Implementation
args.max_iteration = 50 # Maximum iteration num. of algorithm, MAX_STEPS
args.sample_num_per_iter = 50 # Simulation sample num. of each iteration of algorithm
args.experience_num_per_iter = 250 # Experience sample num. of each iteration of algorithm, h
args.replay_batch_num = 16 # Replay batch num., B
args.nn_update_num = 1 # CNN update number, U: [(1) Constant num. of iteration], (2) Lower limit of loss function value
args.batch_size = 32 # Batch size, N
args.replay_memory_size = 1000 # Replay memory size, K

# # For Debugging
# args.max_iteration = 5  # Maximum iteration num. of algorithm, MAX_STEPS
# args.sample_num_per_iter = 3  # Simulation sample num. of each iteration of algorithm
# args.experience_num_per_iter = 15  # Experience sample num. of each iteration of algorithm, h
# args.replay_batch_num = 4  # Replay batch num., B
# args.nn_update_num = 4  # CNN update number, U: [(1) Constant num. of iteration], (2) Lower limit of loss function value
# args.batch_size = 8  # Batch size, N
# args.replay_memory_size = 30  # Replay memory size, K

args.gridnum_x = 15
args.gridnum_y = 15
args.gridsize_x = 120  # ft
args.gridsize_y = 120  # ft

args.time_step = 120  # days
args.total_production_time = 600  # days

args.prod_well_num_max = 5
args.inj_well_num_max = 0
args.total_well_num_max = args.prod_well_num_max + args.inj_well_num_max

args.initial_PRESSURE = 3500  # psi
args.initial_SOIL = 0.75

# 2023-05-02: For reproduction
args.random_seed = 202022673
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

# For calculation of revenue at each time step
args.oil_price = 60  # $/bbl
args.water_treatment = 3  # $/bbl
args.water_injection = 5  # $/bbl

# State: Pressure distribution, Oil saturation, Well placement map

# Action: Well placement (Coordinate of well location)

# Environment: Reservoir simulator

# Reward: NPV at each time segment

args.discount_rate = 0.1  # Used for calculation of NPV
args.discount_factor = 1  # Used for Q-value update

# Data for State
args.input_flag = ('PRESSURE', 'SOIL', 'Well_placement')

In [18]:
################################### Class: Placement sample #################################### Certified
# WellPlacementSample Class contains information of One full sequence of simulation sample.
class WellPlacementSample:
    def __init__(self, args):
        self.args = args
        self.well_loc_map = [[[0 for i in range(0, args.gridnum_x)] for j in range(0, args.gridnum_y)]]
        self.well_loc_list = []
        self.PRESSURE_map = [[[args.initial_PRESSURE for i in range(0, args.gridnum_x)] for j in range(0, args.gridnum_y)]]
        self.SOIL_map = [[[args.initial_SOIL for i in range(0, args.gridnum_x)] for j in range(0, args.gridnum_y)]]
        self.income = []

################################### Class: Experience sample ################################### Certified
# Experience Class contains One set of (s, a, r, s').
class Experience:
    def __init__(self, args):
        self.args = args
        self.current_state = list
        self.current_action = None
        self.reward = None
        self.next_state = list

    def __transform__(self):
        self.current_state = torch.tensor(data=self.current_state, dtype=torch.float, device=self.args.device, requires_grad=False)
        self.current_action = torch.tensor(data=self.current_action, dtype=torch.float, device=self.args.device, requires_grad=False)
        self.reward = torch.tensor(data=self.reward, dtype=torch.float, device=self.args.device, requires_grad=False)
        self.next_state = torch.tensor(data=self.next_state, dtype=torch.float, device=self.args.device, requires_grad=False)

    def transform(self):
        self.__transform__()


###################### Class: Experience sample (Dataset for DataLoader) ####################### Certified
class Experience_list(Dataset):
    def __init__(self, args):
        self.args = args
        self.exp_list = deque()

    def __len__(self):
        return len(self.exp_list)

    def __getitem__(self, idx):
        return self.exp_list[idx].current_state

well_placement_sample = WellPlacementSample(args=args)

# if os.path.exists(os.path.join(args.simulation_directory, f"Step{algorithm_iter_count}_Sample{sample_num}"), f"{args.ecl_filename}_SAM{sample_num}_SEQ{args.total_well_num_max}.PRT"):
if os.path.exists(os.path.join(args.simulation_directory, f"Step1_Sample1", f"{args.ecl_filename}_SAM1_SEQ5.PRT")):
    well_loc_list_from_simulation = []

    with open(os.path.join(args.simulation_directory, f"Step1_Sample1", f"WELL_Sam1_Seq5.DATA")) as file_read:
        line = file_read.readline()
        for time_step in range(0, args.total_well_num_max):
            line = file_read.readline()
            line = file_read.readline()
            line_list = [element.strip() for element in line.split()]
            well_loc = (int(line_list[2]), int(line_list[3]))
            well_loc_list_from_simulation.append(well_loc)
            if time_step == args.total_well_num_max-1:
                break
            while not line.startswith("--WELL"):
                line = file_read.readline()

    for time_step in range(0, args.total_well_num_max):
        well_placement_sample.well_loc_list.append(well_loc_list_from_simulation[time_step])

print(well_placement_sample.well_loc_list)


[(6, 4), (10, 5), (12, 7), (4, 7), (8, 3)]


In [6]:
os.getcwd()
os.chdir("..\\..\\")

In [6]:
os.getcwd()

'J:\\Lab_Meeting\\Simulation\\2D_15-by-15_DQN'

In [14]:
with open(os.path.join(args.simulation_directory, f"Step1_Sample1", f"WELL_Sam1_Seq5.DATA")) as file_read:
    line = file_read.readline()
    line = file_read.readline()
    line = file_read.readline()
    print(line.startswith("--WELL"))
    line_list = [element.strip() for element in line.split()]
    well_loc = (int(line_list[2]), int(line_list[3]))
    print(well_loc)
    while not line.startswith("--WELL"):
        line = file_read.readline()
    print(line)

False
(6, 4)
--WELL #2



In [7]:
print(args.boltzmann_tau_start)

for m in range(1, args.max_iteration + 1):
    # Decrease tau (temperature parameter of Boltzmann policy)
    args.tau = ((args.boltzmann_tau_start - args.boltzmann_tau_end) * np.log(args.max_iteration + 1 - m) + args.boltzmann_tau_end) / ((args.boltzmann_tau_start - args.boltzmann_tau_end) * np.log(args.max_iteration) + args.boltzmann_tau_end) * args.boltzmann_tau_start
    # Tracking variation of tau
    args.boltzmann_tau_tracker.append(args.tau)
    print(args.tau)

8.0
8.0
7.958819162899189
7.9167891768143885
7.8738742842227
7.830036420319526
7.785235010141451
7.739426742888523
7.692565320300054
7.644601175419832
7.595481157468322
7.545148177799024
7.493540811025635
7.440592844331123
7.386232766664588
7.330383187940474
7.2729601764044665
7.2138724999277635
7.15302075401461
7.0902963555980465
7.0255803770459195
6.958742188925109
6.889637872603591
6.81810835420349
6.743977199066561
6.667047989809294
6.5871011899097445
6.503890366724133
6.417137610229272
6.326527932798269
6.2317023653295776
6.132249367708768
6.027694034240869
5.9174843778502195
5.80097368983751
5.677397545507791
5.545843378834856
5.4052095441132355
5.25414917971904
5.090991556633878
4.913629122708015
4.719350557618514
4.504585567759966
4.264498735417537
3.9923107340229813
3.6780927465436237
3.306451747528259
2.851599925327282
2.2651939364533695
1.438701115237028
0.025802305146773764


In [9]:
[1,2,3]/2

TypeError: unsupported operand type(s) for /: 'list' and 'int'